# Vergleich Laufzeit PCA

In [132]:
import pdal 
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import KDTree
from scipy.linalg import svd 
from osgeo import ogr
import pyvista as pv
import os
import json
import time 
import open3d as o3d
from interessant import * # Bei Änderungen Kernel neu starten

ogr.UseExceptions()

In [133]:
bla = []

In [254]:
tmpdir = "/media/riannek/minimax/gleis/temp_fertig"

# Bahnsteig: 29; Gleis hohe Intensität: 11; Weiche B: 16; Unterirdischer Bhf: 20; Gleis weit abseits: 23; Betondeckel: 28; Zug run 14 A (in run24 Achszähler): 6; 
# Viele Gleise: 33; Anfang Weiche: 34; OLA gleiche H: 35; Y: 37

datei_idx = 16 # [0, 1, 16, 33, 23, 34, 16]

key = list(interessant.keys())[datei_idx] 

filename = interessant[key]
filename = filename.split(".")[0] + ".ply"
print(key, filename)


if not os.path.exists(os.path.join(tmpdir, filename)):
    raise FileNotFoundError(filename)

bla.append(datei_idx)
print(bla)

Weiche B 4479025_5352925.ply
[0, 1, 16, 33, 23, 34, 16]


In [255]:
thresh_side_distance = 0.15   # 0.1 ist meist gut, aber in manchen Fällen zu wenig (z.B. Anfang Weiche)

intensity_threshold = 14500
downsample_radius = 0.45 # 0.4
neighborhood_radius = 0.5 # 0.5

min_points = 10
minimum_in_hood = 10
linearity_tresh = 0.98

gauge = 1.435

In [256]:
runfolder = "/media/riannek/minimax/gleis/run24-2024-08-13"

import subprocess
# subprocess.Popen(["pyvistaviewer", os.path.join(runfolder, filename.split(".")[0] + ".copc.laz")])

## Datei incl. margin aus Nachbarkacheln

In [257]:
gpkg = ogr.Open(os.path.join(tmpdir, "temp.gpkg"))
layer = gpkg.GetLayerByName("tiles")

In [258]:
def extend_bbox(bbox, margin=2):
    # MinX, MaxX, MinY, MaxY
    return (bbox[0] - margin, bbox[1] + margin, bbox[2] - margin, bbox[3] + margin)

def get_bbox_polygon(bbox):  
    ring = ogr.Geometry(ogr.wkbLinearRing)      
    ring.AddPoint_2D(bbox[0], bbox[2])  # MinX, MinY
    ring.AddPoint_2D(bbox[1], bbox[2])  # MaxX, MinY
    ring.AddPoint_2D(bbox[1], bbox[3])  # MaxX, MaxY
    ring.AddPoint_2D(bbox[0], bbox[3])  # MinX, MaxY
    ring.AddPoint_2D(bbox[0], bbox[2])  # Close ring
    geom = ogr.Geometry(ogr.wkbPolygon)
    geom.AddGeometry(ring)
    return geom

In [259]:
filter = f"filename = '{filename}'"
layer.SetAttributeFilter(filter)
feature = layer.GetNextFeature()
layer.SetAttributeFilter(None)
bbox = feature.GetGeometryRef().GetEnvelope()
extended = extend_bbox(bbox, margin=2)
bbox_geom = get_bbox_polygon(extended)
layer.SetSpatialFilter(bbox_geom)
tiles = [f.GetField("filename") for f in layer]
layer.SetSpatialFilter(None) 

print(tiles) 

['4479025_5352925.ply', '4479050_5352950.ply', '4479050_5352925.ply', '4479025_5352950.ply', '4479050_5352900.ply', '4479025_5352900.ply', '4479000_5352950.ply', '4479000_5352900.ply']


In [260]:
bbox # MinX, MaxX, MinY, MaxY

(4479025.0, 4479050.0, 5352925.0, 5352950.0)

In [261]:
del gpkg

In [262]:
bounds = f"([{extended[0]}, {extended[1]}], [{extended[2]}, {extended[3]}])" 
bounds

'([4479023.0, 4479052.0], [5352923.0, 5352952.0])'

In [263]:
readers = [pdal.Reader(os.path.join(tmpdir, tile)) for tile in tiles]
pipeline = pdal.Pipeline(readers) | pdal.Filter.merge() | pdal.Filter.crop(bounds=bounds)
pipeline.execute()
points = pipeline.arrays[0]
points.shape 

(105168,)

## Seed Points

In [264]:
low_intensity = points[points["Intensity"] < intensity_threshold]
low_intensity.shape

(75044,)

In [265]:
bounds = f"([{bbox[0]}, {bbox[1]}], [{bbox[2]}, {bbox[3]}])" 
bounds

'([4479025.0, 4479050.0], [5352925.0, 5352950.0])'

In [266]:
# Downsample with poisson sampling (only original bbox)

downsampling_pipeline = pdal.Filter.crop(bounds=bounds).pipeline(low_intensity) | pdal.Filter("filters.sample", radius=downsample_radius)
downsampling_pipeline.execute()
seed_points = downsampling_pipeline.arrays[0]
seed_point_count = seed_points.shape[0]
seed_point_count

284

## KD Tree

In [267]:
xyz = np.vstack((points['X'], points['Y'], points['Z'])).transpose()
xyz_seed = np.vstack((seed_points['X'], seed_points['Y'], seed_points['Z'])).transpose()

In [268]:
offset = xyz.mean(axis=0).round() 
xyz -= offset
xyz_seed -= offset

In [269]:
tree = KDTree(xyz)  

# indices: ndarray (dtype object) with a list of indices for each seed point
indices = tree.query_ball_point(xyz_seed, r=neighborhood_radius)

## Funktionen

In [270]:
def pca_svd(cloud):
    """Use PCA to get einvalues and eigenvectors of a point cloud
    
    Returns (eigenvalues, eigenvectors)
    """
    if cloud.shape[0] < 3:
        raise ValueError("Point cloud must have at least 3 points")
    mean = np.mean(cloud, axis=0)
    centered = cloud - mean
    U, S, Vt = svd(centered, full_matrices=False)
    eigenvals = S**2/(cloud.shape[0]-1)
    # Returned vectors are in columns, first vector is eigenvec[:, 0] == eigenvec.T[0]
    return eigenvals, Vt.T


def pca_cov(cloud):
    """Use PCA to get einvalues and eigenvectors of a point cloud"""
    mean = np.mean(cloud, axis=0)
    centered = cloud - mean
    cov_matrix = np.cov(centered, rowvar=False) # row variance nicht berechnen
    eigenvals, eigenvecs = np.linalg.eig(cov_matrix)
    sorted_indices = np.argsort(eigenvals)[::-1]
    sorted_eigenvals = eigenvals[sorted_indices]
    sorted_eigenvecs = eigenvecs[:,sorted_indices]
    # Returned vectors are in columns, first vector is eigenvec[:, 0] == eigenvec.T[0]
    return sorted_eigenvals, sorted_eigenvecs



In [271]:
def dbscan_stretchz(xyz, min_points=10, stretch=1.5):

    pointcount = xyz.shape[0]
    if pointcount <= min_points:
        return np.ones(pointcount, dtype=np.int8) * -1
    
    eps = 50 / pointcount
    eps = max(eps, 0.06)
    
    xyz = xyz.copy()
    xyz[:, 2] *= stretch
    hood_pcd = o3d.geometry.PointCloud()
    hood_pcd.points = o3d.utility.Vector3dVector(xyz) 
    # eps is min distance between clusters
    labels = np.array(hood_pcd.cluster_dbscan(eps=eps, min_points=min_points, print_progress=False))
    return labels


## Iteration über Seed Points

In [272]:
filename = "pca_speed.csv"

if not os.path.exists(filename):
    f = open(filename, "w")
    f.write("file idx, seed point idx, point count, time cov, time svd\n")
    f.close()


f = open(filename, "a")

In [273]:
for i in range(seed_point_count):
    points = xyz[indices[i]]
    if points.shape[0] < min_points:
        continue
    
    labels = dbscan_stretchz(points, min_points=min_points, stretch=4)
    
    max_label = labels.max()
    
    for label in range(max_label + 1):
        cluster = points[labels == label]
        if cluster.shape[0] < min_points:
            # Somehow this happens, must be a bug in open3d
            continue

        start = time.time()   
        _, _ = pca_cov(cluster)
        time_cov = time.time() - start
        start = time.time()
        _, _ = pca_svd(cluster)
        time_svd = time.time() - start

        f.write(f"{datei_idx}, {i}, {cluster.shape[0]}, {time_cov}, {time_svd}\n")

f.close()

print(f"Wrote {datei_idx}")

Wrote 16
